In [88]:
import pandas as pd 
import numpy as np 
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


In [89]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\singh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\singh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\singh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [103]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                891 non-null    int64  
 1   course_title              891 non-null    object 
 2   course_organization       891 non-null    object 
 3   course_Certificate_type   891 non-null    object 
 4   course_rating             891 non-null    float64
 5   course_difficulty         891 non-null    object 
 6   course_students_enrolled  891 non-null    float64
 7   combined_text             891 non-null    object 
dtypes: float64(2), int64(1), object(5)
memory usage: 55.8+ KB


In [90]:
# # Custom transformer for text cleaning
# class TextCleaner(BaseEstimator, TransformerMixin):
#     def fit(self, X, y=None):
#         return self
    
#     def transform(self, X, y=None):
#         return X.apply(self._clean_text)
    
#     def _clean_text(self, text):
#         text = text.lower()
#         text = re.sub(r'\d+', '', text)  # Remove numbers
#         text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
#         text = text.strip()
#         return text


In [91]:
df = pd.read_csv(r"C:\Users\singh\OneDrive\Documents\Course_rec\cleaned_courses.csv")


In [92]:
from sklearn.base import BaseEstimator, TransformerMixin
import re

class TextCleaner(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.apply(self._clean_text)
    
    def _clean_text(self, text):
        text = text.lower()
        text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
        text = re.sub(r"\s+", " ", text).strip()
        return text


In [93]:
class TextCleaner(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X.apply(self._clean_text)

    def _clean_text(self, text):
        text = text.lower()
        text = re.sub(r'\d+', '', text)
        text = text.translate(str.maketrans('', '', string.punctuation))
        text = text.strip()
        return text

# Create pipeline
pipeline = Pipeline([
    ("cleaner", TextCleaner()),
    ("tfidf", TfidfVectorizer(stop_words="english"))
])

pipeline.fit(df["course_title"])

# ✅ Save pipeline safely
with open("course_pipeline.pkl", "wb") as f:
    pickle.dump(pipeline, f)

In [94]:
# def recommend_courses(query, top_n=5):
#     try:
#         # Clean & transform the input query
#         cleaned_query = course_pipeline.named_steps["cleaner"].transform(pd.Series(query))
#         query_vector = course_pipeline.named_steps["tfidf"].transform(cleaned_query)

#         # Transform the course titles
#         course_texts = course_pipeline.named_steps["cleaner"].transform(df["course_title"])
#         course_vectors = course_pipeline.named_steps["tfidf"].transform(course_texts)

    #     # Compute similarity
    #     sim_scores = cosine_similarity(query_vector, course_vectors).flatten()
    #     top_indices = sim_scores.argsort()[-top_n:][::-1]

    #     # Return the top recommended courses
    #     return df.iloc[top_indices][[
    #         "course_title", 
    #         "course_organization", 
    #         "course_rating", 
    #         "course_difficulty", 
    #         "course_students_enrolled"
    #     ]]
    # except Exception as e:
    #     print(f"❌ Error occurred: {e}")
    #     return pd.DataFrame()


In [101]:
df.head()

,Unnamed: 0,course_title,course_organization,course_Certificate_type,course_rating,course_difficulty,course_students_enrolled,combined_text
0,134,(ISC)² Systems Security Certified Practitioner...,(ISC)²,SPECIALIZATION,4.7,Beginner,5300.0,isc systems security certified practitioner ss...
1,743,A Crash Course in Causality: Inferring Causal...,University of Pennsylvania,COURSE,4.7,Intermediate,17000.0,a crash course in causality inferring causal ...
2,874,A Crash Course in Data Science,Johns Hopkins University,COURSE,4.5,Mixed,130000.0,a crash course in data science johns hopkins u...
3,413,A Law Student's Toolkit,Yale University,COURSE,4.7,Mixed,91000.0,a law students toolkit yale university course ...
4,635,A Life of Happiness and Fulfillment,Indian School of Business,COURSE,4.8,Mixed,320000.0,a life of happiness and fulfillment indian sch...


In [102]:
# pipeline = Pipeline([
#     ("cleaner", TextCleaner()),
#     ("tfidf", TfidfVectorizer(stop_words='english'))
# ])


In [96]:
# pipeline.fit(df["course_title"])


In [97]:
# with open("course_pipeline.pkl", "wb") as f:
#     pickle.dump(pipeline, f)

In [98]:
# def convert_enrolled(x):
#     x = str(x).lower().replace('k', '')
#     try:
#         return float(x) * 1000
#     except:
#         return 0

# df["course_students_enrolled"] = df["course_students_enrolled"].apply(convert_enrolled)


In [100]:
# def clean_text(text):
#     import re
#     text = str(text).lower()
#     text = re.sub(r'[^a-zA-Z\s]', '', text)
#     return text

# df["combined_text"] = (
#     df["course_title"] + " " +
#     df["course_organization"] + " " +
#     df["course_Certificate_type"] + " " +
#     df["course_difficulty"]
# ).apply(clean_text)
